# 1. Import

In [1]:
import pandas as pd
import numpy as np
import uuid
import time
import math

# 2. Constants

In [2]:
eps = 1e-10

# 3. Implementations

## 3.1. Rectangle

In [3]:
class Rectangle:
    __slots__ = ['id', 'x', 'y', 'width', 'height', 'left', 'right', 'bottom', 'top', 'value']
    
    def __init__(self, x, y, height, width, value):
        self.id = uuid.uuid4()
        self.x = x
        self.y = y
        self.width = width
        self.height = height
        self.left = self.x
        self.right = self.x + self.width
        self.bottom = self.y
        self.top = self.y + self.height
        self.value = value
        
        return
    
    def __repr__(self):
        return f'Rectangle(x={self.x}, y={self.y}, width={self.width}, height={self.height}, value={self.value})'
    
    def intersects(self, another):
        return not (self.x + self.width <= another.x or self.x >= another.x + another.width or self.y + self.height <= another.y or self.y >= another.y + another.height)

## 3.2. Genetic Algorithm

In [4]:
class GeneticAlgorithm:
    __slots__ = ['radius', 'init_size', 'population']
    
    def __init__(self, radius, number_of_generations, init_size):
        self.radius = radius
        self.init_size = init_size
        self.population = []
        
        pass
    
    def can_be_placed(self, new_rectangle):        
        # check if new rectangle is strictly inside of circle
        if not (
            np.sqrt(new_rectangle.left ** 2 + new_rectangle.bottom ** 2) < self.radius and
            np.sqrt(new_rectangle.left ** 2 + new_rectangle.top ** 2) < self.radius and
            np.sqrt(new_rectangle.right ** 2 + new_rectangle.bottom ** 2) < self.radius and
            np.sqrt(new_rectangle.right ** 2 + new_rectangle.top ** 2) < self.radius
        ):
            return False

        # check if new rectangle intersects with any other
        for rectangle in self.population:
            if new_rectangle.intersects(rectangle):
                return False
            
        return True
    
    def generate_rectangle(self, width, height, value, x=None, y=None):
        # generate x
        if x is None:
            x = np.random.rand() * 2 * self.radius - self.radius

        # generate y based on x
        if y is None:
            y = np.random.rand() * 2 * np.sqrt(self.radius ** 2 - x ** 2) - np.sqrt(self.radius ** 2 - x ** 2)
        
        # create new rectangle
        return Rectangle(x, y, width, height, value)
    
    def mutation(self, creature):
        # moving top
        collisions_while_vertical_move = []
        for rect in self.population:
            if not (rect.right - eps <= creature.left or creature.right - eps <= rect.left):
                collisions_while_vertical_move.append(rect)
        
        last_available_bottom = float("inf")
        for rect in collisions_while_vertical_move:            
            if rect.bottom > creature.top - eps and rect.bottom - eps < last_available_bottom:
                last_available_bottom = rect.bottom
                
        last_available_bottom = min(last_available_bottom, np.sqrt(self.radius ** 2 - creature.left ** 2), np.sqrt(self.radius ** 2 - creature.right ** 2))
        
        if last_available_bottom > creature.top - eps:
            creature.y = creature.bottom = last_available_bottom - creature.height
            creature.top = creature.bottom + creature.height
        
        # moving right
        collisions_while_horizontal_move = []
        for rect in self.population:
            if not (rect.top - eps <= creature.bottom or creature.top - eps <= rect.bottom):
                collisions_while_horizontal_move.append(rect)
        
        last_available_left = float("inf")
        for rect in collisions_while_horizontal_move:            
            if rect.left > creature.right - eps and rect.left - eps < last_available_left:
                last_available_left = rect.left
                
        last_available_left = min(last_available_left, np.sqrt(self.radius ** 2 - creature.bottom ** 2), np.sqrt(self.radius ** 2 - creature.top ** 2))
        
        if last_available_left > creature.right - eps:
            creature.x = creature.left = last_available_left - creature.width
            creature.right = creature.left + creature.width
    
    def optimize(self):
        # read data and use only values
        df = pd.read_csv(f'../data/r{self.radius}.csv', header=None).values
        # allow to swap width and height values
        df = np.append(df, df[:, [1,0,2]], axis=0)
        # retrieve only unique entries
        df = np.unique(df, axis=0)
        # sort in accordance with the highest value per area
        df = df[np.flip(np.argsort(df[:, 2] / (df[:, 0] * df[:, 1])))]
        
        # for every type of rectangle from the one with the highest value per area to the lowest
        for i in range(len(df)):
            # try to create init_size rectangles of such type
            for _ in range(self.init_size):
                new_rect = self.generate_rectangle(*(df[i]))
                if self.can_be_placed(new_rect):
                    self.population.append(new_rect)
                    self.mutation(new_rect)
        
        return self.population

# 4. Stuff for verification

## 4.1. Conveniently display elapsed time

In [5]:
def seconds_to_time(seconds):
    diff = math.ceil(seconds)
    hours = diff // 3600
    minutes = (diff - hours * 3600) // 60
    seconds = diff - hours * 3600 - minutes * 60
    
    return hours, minutes, seconds

## 4.2. Enable display of a result

In [6]:
def to_html(population, radius):
    rects = ''
    for rect in population:
        rects += f'<rect x="{rect.x}" y="{rect.y}" width="{rect.width}" height="{rect.height}"></rect>'
    
    return f'''<svg height="720px" viewBox="-{radius} -{radius} {2 * radius} {2 * radius}" transform="scale(1,-1)"><circle cx="0" cy="0" r="{radius}" fill="white" stroke="red" stroke-width="5px"></circle>{rects}</svg>'''

## 4.3. Expected scores per circle radius

In [7]:
expected_output = {
    800: 30000,
    850: None,
    1000: 17500,
    1100: 25000,
    1200: 30000
}

## 4.4. Function to present results

In [8]:
def solution(radius, trials):
    best_score = 0
    best_time = 0
    best_solution = []
    
    total_score = 0
    total_time = 0
    
    for _ in range(trials):
        start = time.time()
        ga = GeneticAlgorithm(radius, 50000, 1000)
        final_population = ga.optimize()
        score = 0
        for rect in final_population:
            score += rect.value
        end = time.time()
        
        if score > best_score:
            best_score = score
            best_time = end - start
            best_solution = final_population
        
        total_score += score
        total_time += end - start
        
    avg_score = total_score / trials
    avg_time = total_time / trials
    
    hours, minutes, seconds = seconds_to_time(best_time)
    avg_hours, avg_minutes, avg_seconds = seconds_to_time(avg_time)
    
    print('best solution html svg representation')
    print(f'{to_html(best_solution, radius)}')
    print()
    print(f'model evaluated on average in {avg_hours}h {avg_minutes}m {avg_seconds}s')
    print(f'average score: {avg_score}')
    print()
    print(f'best score evaluated in {hours}h {minutes}m {seconds}s')
    print(f'best score: {best_score}')
    print(f'expected score: {expected_output[radius]}')

# 5. Results

## 5.1. Dataset with radius 800

In [9]:
solution(800, 50)

best solution html svg representation
<svg height="720px" viewBox="-800 -800 1600 1600" transform="scale(1,-1)"><circle cx="0" cy="0" r="800" fill="white" stroke="red" stroke-width="5px"></circle><rect x="582.546578989005" y="114.57427896161573" width="30" height="400"></rect><rect x="296.1459922610844" y="330.49900186929256" width="30" height="400"></rect><rect x="170.2160612014047" y="374.54085033456784" width="30" height="400"></rect><rect x="666.842393293045" y="-7.048757592470281" width="30" height="400"></rect><rect x="510.8962941095872" y="189.43294700798242" width="30" height="400"></rect><rect x="445.38259542514567" y="243.4371670698331" width="30" height="400"></rect><rect x="93.60218465200127" y="390.39388911431536" width="30" height="400"></rect><rect x="63.60218465200127" y="306.5557433225805" width="30" height="400"></rect><rect x="552.546578989005" y="-210.56705299201758" width="30" height="400"></rect><rect x="33.60218465200127" y="396.2692180526784" width="30" height="

## 5.2. Dataset with radius 850

In [10]:
solution(850, 50)

best solution html svg representation
<svg height="720px" viewBox="-850 -850 1700 1700" transform="scale(1,-1)"><circle cx="0" cy="0" r="850" fill="white" stroke="red" stroke-width="5px"></circle><rect x="472.432685181128" y="196.00164732264318" width="80" height="450"></rect><rect x="636.2468246898011" y="7.701306663829996" width="80" height="450"></rect><rect x="-10.420552918364407" y="397.14739009443565" width="80" height="450"></rect><rect x="556.2468246898011" y="-253.99835267735682" width="80" height="450"></rect><rect x="-90.42055291836441" y="385.22617784687327" width="80" height="450"></rect><rect x="70.9781269909748" y="386.4840734707966" width="80" height="450"></rect><rect x="-170.42055291836442" y="313.86652563517305" width="80" height="450"></rect><rect x="-250.42055291836442" y="296.6918050512528" width="80" height="450"></rect><rect x="-330.4205529183644" y="231.65711870195992" width="80" height="450"></rect><rect x="392.432685181128" y="-218.34288129804008" width="80" 

## 5.3. Dataset with radius 1000

In [11]:
solution(1000, 50)

best solution html svg representation
<svg height="720px" viewBox="-1000 -1000 2000 2000" transform="scale(1,-1)"><circle cx="0" cy="0" r="1000" fill="white" stroke="red" stroke-width="5px"></circle><rect x="30.8860837060229" y="731.6121958530147" width="160" height="250"></rect><rect x="459.1226963735653" y="535.29426766985" width="160" height="250"></rect><rect x="681.5698042143331" y="290.14837279644655" width="160" height="250"></rect><rect x="299.1226963735653" y="392.6331669174016" width="160" height="250"></rect><rect x="796.9816726372425" y="40.14837279644655" width="160" height="250"></rect><rect x="-129.1139162939771" y="629.0535955649357" width="160" height="250"></rect><rect x="-289.1139162939771" y="695.7554407863732" width="160" height="250"></rect><rect x="139.1226963735653" y="379.0535955649357" width="160" height="250"></rect><rect x="-449.1139162939771" y="611.326452132847" width="160" height="250"></rect><rect x="-609.1139162939771" y="513.4777218248585" width="160" 

## 5.4. Dataset with radius 1100

In [12]:
solution(1100, 50)

best solution html svg representation
<svg height="720px" viewBox="-1100 -1100 2200 2200" transform="scale(1,-1)"><circle cx="0" cy="0" r="1100" fill="white" stroke="red" stroke-width="5px"></circle><rect x="226.92501377903693" y="779.7033717105583" width="160" height="250"></rect><rect x="-26.7283906577577" y="841.8968257776596" width="160" height="250"></rect><rect x="-186.7283906577577" y="818.6546859077091" width="160" height="250"></rect><rect x="615.9269631500035" y="529.7033717105583" width="160" height="250"></rect><rect x="-346.7283906577577" y="782.303609713251" width="160" height="250"></rect><rect x="804.0613766760216" y="279.70337171055826" width="160" height="250"></rect><rect x="455.92696315000353" y="529.7033717105583" width="160" height="250"></rect><rect x="66.92501377903693" y="532.303609713251" width="160" height="250"></rect><rect x="295.92696315000353" y="529.7033717105583" width="160" height="250"></rect><rect x="644.0613766760216" y="73.44642424724759" width="16

## 5.5. Dataset with radius 1200

In [13]:
solution(1200, 50)

best solution html svg representation
<svg height="720px" viewBox="-1200 -1200 2400 2400" transform="scale(1,-1)"><circle cx="0" cy="0" r="1200" fill="white" stroke="red" stroke-width="5px"></circle><rect x="735.2228894024497" y="549.109490802061" width="160" height="250"></rect><rect x="280.5282199643781" y="866.2145346728901" width="160" height="250"></rect><rect x="575.2228894024497" y="405.99295974329584" width="160" height="250"></rect><rect x="79.78142016362898" y="925.7996727947807" width="160" height="250"></rect><rect x="120.52821996437808" y="623.9736107456299" width="160" height="250"></rect><rect x="415.2228894024497" y="616.2145346728901" width="160" height="250"></rect><rect x="-80.21857983637102" y="841.0066399772886" width="160" height="250"></rect><rect x="-39.47178003562192" y="591.0066399772886" width="160" height="250"></rect><rect x="415.2228894024497" y="366.21453467289007" width="160" height="250"></rect><rect x="-199.47178003562192" y="591.0066399772886" width="